# Check if imaging analysis has run

In [2]:
import os
import glob
import pandas as pd

def load_list(fn):
    return [str(x) for x in open( fn ).read().split('\n')[:-1] ]

def to_list(in_list, fn):
    with open(fn, "w") as file:
        for row in in_list:
            file.write(str(row)+'\n')

In [4]:
cfs_path = "/data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/"
workspace_path = "/data/workspaces/lag/workspaces/lg-ukbiobank/projects/FLICA_multimodal/"

check_subs = True
successful = []
ifl_only = []
mtl_only = []
started = []
no_diff = []
no_tbm = []
not_started = []

subs_in_file1 = open(os.path.join(workspace_path, "participant_list_imaging_sex_check_50k.txt"))
subs_in = [int(x) for x in sorted(list(subs_in_file1.read().split('\n')))[1:]]
#subs_in_file2 = open(os.path.join(workspace_path, "participants_thousand.txt"))
#subs_in = subs_in + [int(x) for x in sorted(list(subs_in_file2.read().split('\n')))[1:]]
#subs_in_file3 = open(os.path.join(workspace_path, "participants_ten_thousand.txt"))
#subs_in = subs_in + [int(x) for x in sorted(list(subs_in_file3.read().split('\n')))[1:]]
#subs_in_file4 = open(os.path.join(workspace_path, "participants_next_ten_thousand.txt"))
#subs_in = subs_in + [int(x) for x in sorted(list(subs_in_file4.read().split('\n')))[1:]]
#files_native = sorted(glob.glob(os.path.join(cfs_path,"dr_stage2_native", "*", "ica_ifl_standard.nii.gz")))
#subs_native = set([ int(os.path.split(os.path.split(x)[-2])[-1]) for x in files_native])
#folders_mni = sorted(glob.glob(os.path.join(cfs_path,"dr_stage2", "*")))
#subs_folders = set([ int(os.path.split(x)[-1]) for x in folders_mni])

files_ifl = sorted(glob.glob(os.path.join(cfs_path, "dr_stage2", "*", "ica_ifl.nii.gz")))
files_mtl = sorted(glob.glob(os.path.join(cfs_path, "dr_stage2", "*", "ica_mtl.nii.gz")))
files_diff = sorted(glob.glob("/data/clusterfs/lag/projects/lg-ukbiobank/working_data/qc-sourena/sw-preproc/*/*-afd.nii.gz"))
#files_diff = sorted(glob.glob("/data/clusterfs/lag/projects/lg-ukbiobank/primary_data/imaging_data/*/dMRI/dMRI/dti_FA.nii.gz"))
files_tbm = sorted(glob.glob("/data/clusterfs/lag/projects/lg-ukbiobank/working_data/qc-sourena/sw-preproc/*/*-unbiased-brain-composite-warp-jac-roi_iso_2.nii.gz"))
subs_ifl = set([ int(os.path.split(os.path.split(x)[-2])[-1]) for x in files_ifl])
subs_mtl = set([ int(os.path.split(os.path.split(x)[-2])[-1]) for x in files_mtl])
#subs_diff = set([ int(os.path.split(os.path.split(os.path.split(os.path.split(x)[-2])[-2])[-2])[-1]) for x in files_diff])
subs_diff = set([ int(os.path.split(os.path.split(x)[-2])[-1]) for x in files_diff])
subs_tbm = set([ int(os.path.split(os.path.split(x)[-2])[-1]) for x in files_tbm])



if check_subs:
    for sub in subs_in:
        if sub in subs_ifl:
            if sub in subs_mtl:
                if sub in subs_diff:
                    if sub in subs_tbm:
                        successful.append(sub)
                    else:
                        no_tbm.append(sub)
                else:
                    no_diff.append(sub)
            else:
                ifl_only.append(sub)
        elif sub in subs_mtl:
            mtl_only.append(sub)
        elif os.path.isfile(os.path.join(cfs_path, "dr_stage2", str(sub))):
            started.append(sub)
        else:
            not_started.append(sub)

    print("Successful: {}".format(len(successful)))
    print("IFL only: {}".format(len(ifl_only)))
    print("MTL only: {}".format(len(mtl_only)))
    print("No diff: {}".format(len(no_diff)))
    print("No tbm: {}".format(len(no_tbm)))
    print("Started: {}".format(len(started)))
    print("Not started: {}".format(len(not_started)))
#print("Subjects started, not finished:")
#print(not_started)

#to_list(fn=os.path.join(workspace_path, "missing_diff.txt"),
#       in_list=not_started)

Successful: 28167
IFL only: 0
MTL only: 0
No diff: 4571
No tbm: 0
Started: 0
Not started: 18


In [31]:
to_list(fn=os.path.join(workspace_path, "missing_resting_state_newest.txt"),
       in_list=not_started)

In [5]:
to_list(fn=os.path.join(workspace_path, "missing_diff_N{}.txt".format(len(no_tbm))),
       in_list=no_diff)

In [17]:
diff_data_zip = list(glob.glob("/data/workspaces/lag/workspaces/lg-ukbiobank/primary_data/imaging_data/*/*_20250_2_0.zip"))
diff_zip_set = set([int(x[-21:-14]) for x in diff_data_zip])

print("Number of diffusion files on workspace: {}".format(len(diff_zip_set)))

diff_data_cfs = list(glob.glob("/data/clusterfs/lag/projects/lg-ukbiobank/primary_data/imaging_data/*/dMRI/dMRI/data.eddy_rotated_bvecs"))
diff_cfs_set = set([int(x[-41:-34]) for x in diff_data_cfs])

print("Number of diffusion files on cfs: {}".format(len(diff_cfs_set)))

on_cluster = []
on_workspace= []
not_available = []

for sub in no_diff:
    if sub in diff_cfs_set:
        on_cluster.append(sub)
    elif sub in diff_zip_set:
        on_workspace.append(sub)
    else:
        not_available.append(sub)
        
print("Missing diffusion subs: {}".format(len(no_diff)))
print("Missing diffusion subs on cluster: {}".format(len(on_cluster)))
print("Missing diffusion subs on workspace: {}".format(len(on_workspace)))
print("Missing diffusion subs unavailable: {}".format(len(not_available)))

Number of diffusion files on workspace: 47557
Number of diffusion files on cfs: 35240
Missing diffusion subs: 4571
Missing diffusion subs on cluster: 4124
Missing diffusion subs on workspace: 445
Missing diffusion subs unavailable: 2


In [18]:
to_list(fn=os.path.join(workspace_path, "missing_diff_on_cluster_N{}.txt".format(len(on_cluster))),
       in_list=on_cluster)
to_list(fn=os.path.join(workspace_path, "missing_diff_on_workspace_N{}.txt".format(len(on_workspace))),
       in_list=on_workspace)

In [3]:
def folder_size(path='.'):
    total = 0
    for entry in os.scandir(path):
        if entry.is_file():
            total += entry.stat().st_size
        elif entry.is_dir():
            total += folder_size(entry.path)
    return total
folder_sz_Gb_fmri = folder_size(os.path.join(cfs_path, "fMRI")) / (1024*1024*1024)
folder_sz_Gb_dr2 =  folder_size(os.path.join(cfs_path, "dr_stage2")) / (1024*1024*1024)
print("Current storage size after running {0} subjects (also includes some other files in directory):".format(len(subs_in)))
print("{1} Gb for raw fMRI files and {2} Gb for dr stage 2 output".format(len(successful), folder_sz_Gb_fmri, folder_sz_Gb_dr2))
#print("Estimated storage size: {}".format((folder_sz_Gb_dr2)*1.5))

Current storage size after running 30878 subjects (also includes some other files in directory):
31.49293038342148 Gb for raw fMRI files and 2468.264628571458 Gb for dr stage 2 output


In [7]:
def folder_size(path='.'):
    total = 0
    for entry in os.scandir(path):
        if entry.is_file():
            total += entry.stat().st_size
        elif entry.is_dir():
            total += folder_size(entry.path)
    return total
cfs_path = "/data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/"
folder_sz_Gb_tbss = folder_size(os.path.join(cfs_path, "tbss")) / (1024*1024*1024)
#folder_sz_Gb_dr2 =  folder_size(os.path.join(cfs_path, "dr_stage2")) / (1024*1024*1024)
print("Current storage size after running {0} subjects (also includes some other files in directory):".format(len(subs_in)))
print("{1} Gb for running TBSS".format(len(successful), folder_sz_Gb_tbss))
print("Estimated storage size: {}".format((folder_sz_Gb_tbss)*285))

Current storage size after running 30878 subjects (also includes some other files in directory):
1.2901520822197199 Gb for running TBSS
Estimated storage size: 367.69334343262017


In [4]:
successful = sorted(successful)

ifl_files = ["/data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/dr_stage2/{}/ica_ifl.nii.gz".format(x) for x in successful]
mtl_files = ["/data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/dr_stage2/{}/ica_mtl.nii.gz".format(x) for x in successful]
tbm_files = ["/data/clusterfs/lag/projects/lg-ukbiobank/working_data/qc-sourena/sw-preproc/{0}/{0}-unbiased-brain-composite-warp-jac-roi_iso_2.nii.gz".format(x) for x in successful]
diff_files = ["/data/clusterfs/lag/projects/lg-ukbiobank/working_data/qc-sourena/sw-preproc/{0}/{0}-afd.nii.gz".format(x) for x in successful]
#fa_files = ["/data/clusterfs/lag/projects/lg-ukbiobank/primary_data/imaging_data/{0}/dMRI/dMRI/dti_FA.nii.gz".format(x) for x in successful]
#md_files = ["/data/clusterfs/lag/projects/lg-ukbiobank/primary_data/imaging_data/{0}/dMRI/dMRI/dti_MD.nii.gz".format(x) for x in successful]

to_list(fn=os.path.join(workspace_path, "ifl_files_in_N{}.txt".format(len(ifl_files))),
       in_list=ifl_files)
to_list(fn=os.path.join(workspace_path, "mtl_files_in_N{}.txt".format(len(mtl_files))),
       in_list=mtl_files)
to_list(fn=os.path.join(workspace_path, "tbm_files_in_N{}.txt".format(len(tbm_files))),
       in_list=tbm_files)
#to_list(fn=os.path.join(workspace_path, "dmri_files_in_N{}.txt".format(len(diff_files))),
#       in_list=diff_files)
#to_list(fn=os.path.join(workspace_path, "fa_files_in_N{}.txt".format(len(fa_files))),
#       in_list=fa_files)
#to_list(fn=os.path.join(workspace_path, "md_files_in_N{}.txt".format(len(md_files))),
#       in_list=md_files)

In [6]:
to_list(fn=os.path.join(workspace_path, "pre_flica_subs_N{}.txt".format(len(successful))),
       in_list=successful)

In [20]:
missing = []
for fn in diff_files:
    if os.path.isfile(fn):
        pass
    else:
        missing.append(fn)
print(len(missing))

1855


In [13]:
#OLD
#fslqc:
mkdir -p  /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/flica_in/rs_ifl
fslmerge -t /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/flica_in/rs_ifl/ica_ifl_mni_1.nii.gz $(cat /data/workspaces/lag/workspaces/lg-ukbiobank/projects/FLICA_multimodal/ifl_files_in_N26660.txt | head -n 10000)
fslmerge -t /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/flica_in/rs_ifl/ica_ifl_mni_2.nii.gz $(cat /data/workspaces/lag/workspaces/lg-ukbiobank/projects/FLICA_multimodal/ifl_files_in_N26660.txt | head -n 20000 | tail -n 10000)
fslmerge -t /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/flica_in/rs_ifl/ica_ifl_mni_3.nii.gz $(cat /data/workspaces/lag/workspaces/lg-ukbiobank/projects/FLICA_multimodal/ifl_files_in_N26660.txt | tail -n 6660)
fslmerge -t /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/flica_in/rs_ifl/ica_ifl_mni.nii.gz /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/flica_in/rs_ifl/ica_ifl_mni_*.nii.gz 
rm /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/flica_in/rs_ifl/ica_ifl_mni_*.nii.gz 

#fslmerge -t /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/flica_in/rs_ifl/ica_ifl_mni.nii.gz $(cat /data/workspaces/lag/workspaces/lg-ukbiobank/projects/FLICA_multimodal/ifl_files_in_N28515.txt)
fslmaths /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/flica_in/rs_ifl/ica_ifl_mni.nii.gz -Tstd /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/dr_stage2/ica_ifl_mni_std.nii.gz
fslmaths /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/flica_in/rs_ifl/ica_ifl_mni.nii.gz -Tmean /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/dr_stage2/ica_ifl_mni_mean.nii.gz
fslmaths /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/flica_in/rs_ifl/ica_ifl_mni.nii.gz -Tmin /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/dr_stage2/ica_ifl_mni_min.nii.gz
fslmaths /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/flica_in/rs_ifl/ica_ifl_mni.nii.gz -Tmax /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/dr_stage2/ica_ifl_mni_max.nii.gz
 

93


In [ ]:
mkdir -p  /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/flica_in/rs_mtl
fslmerge -t /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/flica_in/rs_mtl/ica_mtl_mni_1.nii.gz $(cat /data/workspaces/lag/workspaces/lg-ukbiobank/projects/FLICA_multimodal/mtl_files_in_N26660.txt | head -n 10000)
fslmerge -t /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/flica_in/rs_mtl/ica_mtl_mni_2.nii.gz $(cat /data/workspaces/lag/workspaces/lg-ukbiobank/projects/FLICA_multimodal/mtl_files_in_N26660.txt | head -n 20000 | tail -n 10000)
fslmerge -t /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/flica_in/rs_mtl/ica_mtl_mni_3.nii.gz $(cat /data/workspaces/lag/workspaces/lg-ukbiobank/projects/FLICA_multimodal/mtl_files_in_N26660.txt | tail -n 6660)
fslmerge -t /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/flica_in/rs_mtl/ica_mtl_mni.nii.gz /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/flica_in/rs_mtl/ica_mtl_mni_*.nii.gz 
rm /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/flica_in/rs_mtl/ica_mtl_mni_*.nii.gz 

fslmaths /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/flica_in/rs_mtl/ica_mtl_mni.nii.gz -Tstd /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/dr_stage2/ica_mtl_mni_std.nii.gz
fslmaths /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/flica_in/rs_mtl/ica_mtl_mni.nii.gz -Tmean /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/dr_stage2/ica_mtl_mni_mean.nii.gz
fslmaths /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/flica_in/rs_mtl/ica_mtl_mni.nii.gz -Tmin /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/dr_stage2/ica_mtl_mni_min.nii.gz
fslmaths /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/flica_in/rs_mtl/ica_mtl_mni.nii.gz -Tmax /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/dr_stage2/ica_mtl_mni_max.nii.gz

In [ ]:
mkdir -p  /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/flica_in/tbm
fslmerge -t /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/flica_in/tbm/tbm_1.nii.gz $(cat /data/workspaces/lag/workspaces/lg-ukbiobank/projects/FLICA_multimodal/tbm_files_in_N26660.txt | head -n 10000)
fslmerge -t /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/flica_in/tbm/tbm_2.nii.gz $(cat /data/workspaces/lag/workspaces/lg-ukbiobank/projects/FLICA_multimodal/tbm_files_in_N26660.txt | head -n 20000 | tail -n 10000)
fslmerge -t /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/flica_in/tbm/tbm_3.nii.gz $(cat /data/workspaces/lag/workspaces/lg-ukbiobank/projects/FLICA_multimodal/tbm_files_in_N26660.txt | tail -n 6660)
fslmerge -t /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/flica_in/tbm/tbm.nii.gz /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/flica_in/tbm/tbm_*.nii.gz
rm /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/flica_in/tbm/tbm_*.nii.gz

fslmaths /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/flica_in/tbm/tbm.nii.gz -Tstd /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/dr_stage2/tbm_std.nii.gz
fslmaths /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/flica_in/tbm/tbm.nii.gz -Tmean /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/dr_stage2/tbm_mean.nii.gz
fslmaths /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/flica_in/tbm/tbm.nii.gz -Tmin /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/dr_stage2/tbm_min.nii.gz
fslmaths /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/flica_in/tbm/tbm.nii.gz -Tmax /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/dr_stage2/tbm_max.nii.gz

In [ ]:
mkdir -p  /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/flica_in/dMRI
fslmerge -t /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/flica_in/dMRI/dMRI_1.nii.gz $(cat /data/workspaces/lag/workspaces/lg-ukbiobank/projects/FLICA_multimodal/dmri_files_in_N26660.txt | head -n 10000)
fslmerge -t /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/flica_in/dMRI/dMRI_2.nii.gz $(cat /data/workspaces/lag/workspaces/lg-ukbiobank/projects/FLICA_multimodal/dmri_files_in_N26660.txt | head -n 20000 | tail -n 10000)
fslmerge -t /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/flica_in/dMRI/dMRI_3.nii.gz $(cat /data/workspaces/lag/workspaces/lg-ukbiobank/projects/FLICA_multimodal/dmri_files_in_N26660.txt | tail -n 6660)
fslmerge -t /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/flica_in/dMRI/dMRI.nii.gz /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/flica_in/dMRI/dMRI_*.nii.gz
rm /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/flica_in/dMRI/dMRI_*.nii.gz

fslmaths /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/flica_in/dMRI/dMRI.nii.gz -Tstd /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/dr_stage2/dMRI_std.nii.gz
fslmaths /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/flica_in/dMRI/dMRI.nii.gz-Tmean /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/dr_stage2/dMRI_mean.nii.gz
fslmaths /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/flica_in/dMRI/dMRI.nii.gz -Tmin /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/dr_stage2/dMRI_min.nii.gz
fslmaths /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/flica_in/dMRI/dMRI.nii.gz -Tmax /data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/dr_stage2/dMRI_max.nii.gz